In [1]:
from ete3 import EvolTree

tree = EvolTree("((Hylobates_lar,(Gorilla_gorilla,Pan_troglodytes)),Papio_cynocephalus);")

tree.link_to_alignment ('''>Hylobates_lar
ATGGCCAGGTACAGATGCTGCCGCAGCCAGAGCCGGAGCAGATGTTACCGCCAGAGCCGGAGCAGATGTTACCGCCAGAGGCAAAGCCAGAGTCGGAGCAGATGTTACCGCCAGAGCCAGAGCCGGAGCAGATGTTACCGCCAGAGACAAAGAAGTCGGAGACGAAGGAGGCGGAGCTGCCAGACACGGAGGAGAGCCATGAGGTGT---CGCCGCAGGTACAGGCTGAGACGTAGAAGCTGTTACCACATTGTATCT
>Papio_cynocephalus
ATGGCCAGGTACAGATGCTGCCGCAGCCAGAGCCGAAGCAGATGCTATCGCCAGAGCCGGAGCAGATGTAACCGCCAGAGACAGAGCCAAAGCCGGAGAAGCTGCTATCGCCAGAGCCAAAGCCGGAGCAGATGTTACCGCCAGAGACAGAGAAGTCGTAGACGAAGGAGGCGACGCTGCCAGACACGGAGGAGAGCCATGAGGTGCTTCCGCCGCAGGTACAGGCTGAGGCGTAGGAGGCCCTATCACATCGTGTCT
>Gorilla_gorilla
ATGGCCAGGTACAGATGCTGTCGCAGCCAGAGCCGCAGCAGATGTTACCGGCAGAGCCGGAGCAGGTGTTACCGGCAGAGACAAAGCCAGAGCCGGAGCAGATGCTACCGGCAGAGCCAAAGCCGGAGCAGGTGTTACCGGCAGAGACAAAGAAGTCGCAGACGTAGGCGGAGGAGCTGCCAGACACGGAGGAGAGCCATGAGGTGCTGCCGCCGCAGGTACAGACTGAGACGTAGAAGACCCTATCATATTGTATCT
>Pan_troglodytes
ATGGCCAGGTACAGATGCTGTCGCAGCCAGAGCCGGAGCAGATGTTACCGGCAGAGACGGAGCAGGTGTTACCGGCAAAGGCAAAGCCAAAGTCGGAGCAGATGTTACCGGCAGAGCCAGAGACGGAGCAGGTGTTACCGGCAAAGACAAAGAAGTCGCAGACGAAGGCGACGGAGCTGCCAGACACGGAGGAGAGCCATGAGGTGCTGCCGCCGCAGGTACAGACTGAGACGTAAAAGATGTTACCATATTGTATCT''')

tree.workdir = '/Users/jra/bin/testing'

In [4]:
best_model = None
best_lnl   = float('-inf')
for starting_omega in [0.2, 0.7, 1.2]:
    tree.run_model('fb.'+str(starting_omega))
    current_model = tree.get_evol_model('fb.'+str(starting_omega))
    print('current model log likelihood:', current_model.lnL)
    if current_model.lnL > best_lnl:
        best_lnl = current_model.lnL
        best_model = current_model

current model log likelihood: -521.421323
current model log likelihood: -521.421323
current model log likelihood: -521.421323


In [10]:
tree.run_model('M1')
tree.run_model('M2')

pval = tree.get_most_likely('M2','M1')

model2 = tree.get_evol_model('M2')

print(model2)

# Evolutionary Model M2:
#        log likelihood       : -517.824749
#        number of parameters : 11
#        sites inference      : BEB, NEB
#        sites classes        :
#           proportions: p0=0.98794   p1=0.00000   p2=0.01206
#           w          : w0=0.09887   w1=1.00000   w2=178.86153
#        branches             :
#           mark: #0, omega: 2.25526440485   , nodes paml_ids: 6 4 2 7 1 3

if pval < 0.05:
    print('M2 model wins.')
    for s in range(len(model2.sites['BEB']['aa'])):
        if model2.sites['BEB']['p2'][s] > 0.95:
            print('positively selected site %s at position: %s, with probability: %s' % (model2.sites['BEB']['aa'][s], s+1, model2.sites['BEB']['p2'][s]))
else:
    print('M1 model is not rejected')

# M2 model wins.
# positively selected site P at position: 81, with probability: 0.96293

 Evolutionary Model M2:
        log likelihood       : -517.824749
        number of parameters : 11
        sites inference      : BEB, NEB
        sites classes        : 
        proportions : p0=0.98794   p1=0.0       p2=0.01206   
        w           : w0=0.09887   w1=1.0       w2=178.86135 
        branches             : 
        mark:     , omega: None      , node_ids: 5   , name: ROOT
        mark:     , omega: 2.2553    , node_ids: 6   , name: EDGE
        mark:     , omega: 2.2553    , node_ids: 4   , name: Papio_cynocephalus
        mark:     , omega: 2.2553    , node_ids: 2   , name: Hylobates_lar
        mark:     , omega: 2.2553    , node_ids: 7   , name: EDGE
        mark:     , omega: 2.2553    , node_ids: 1   , name: Gorilla_gorilla
        mark:     , omega: 2.2553    , node_ids: 3   , name: Pan_troglodytes
        
M2 model wins.
positively selected site P at position: 81, with probability: 0.96293


In [14]:
tree.run_model('M0')

for leaf in tree:
    leaf.node_id
    print('\n---------\nNow working with leaf ' + leaf.name)
    tree.mark_tree([leaf.node_id], marks=['#1'])
    print(tree.write())
    # To organize a bit, we name the model with the name of the marked node
    # Any character after the dot, in the model name, is not taken into account
    # for computation. (have a look in /tmp/ete3.../bsA.. directory)
    print('running model bsA and bsA1')
    tree.run_model('bsA.' + leaf.name)
    tree.run_model('bsA1.' + leaf.name)
    print('p-value of positive selection for sites on this branch is:')
    ps = tree.get_most_likely('bsA.' + leaf.name, 'bsA1.' + leaf.name)
    rx = tree.get_most_likely('bsA1.' + leaf.name, 'M0')
    print(str(ps))
    print('p-value of relaxation for sites on this branch is:')
    print(str(rx))
    if ps < 0.05 and float(bsA.wfrg2a) > 1:
        print('we have positive selection on sites on this branch')
    elif rx < 0.05 and ps >= 0.05:
        print('we have relaxation on sites on this branch')
    else:
        print('no signal detected on this branch, best fit for M0')
    print('\nclean tree, remove marks')
    tree.mark_tree(list(map(lambda x: x.node_id, tree.get_descendants())),
                    marks=[''] * len(tree.get_descendants()), verbose=True)

print(tree.get_evol_model('bsA.Hylobates_lar'))



---------
Now working with leaf Hylobates_lar
((Hylobates_lar #1,(Gorilla_gorilla,Pan_troglodytes)),Papio_cynocephalus);
running model bsA and bsA1
p-value of positive selection for sites on this branch is:
0.9327919343756689
p-value of relaxation for sites on this branch is:
0.010403125387005385
we have relaxation on sites on this branch

clean tree, remove marks

---------
Now working with leaf Gorilla_gorilla
((Hylobates_lar ,(Gorilla_gorilla #1,Pan_troglodytes ) ) ,Papio_cynocephalus );
running model bsA and bsA1


/Users/jra/miniconda3/envs/ete3/lib/python3.6/site-packages/ete3/evol/evoltree.py:417: UserWarning: WARNING: marks should be "#" sign directly followed by integer

        function to mark branches on tree in order that paml could interpret it.
        takes a "marks" argument that should be a list of #1,#1,#2
        e.g.:
        ::

          t=Tree.mark_tree([2,3], marks=["#1","#2"])

        :argument node_ids: list of node ids (have a look to node.node_id)
        :argument False verbose: warn if marks do not correspond to codeml standard
        :argument kargs: mainly for the marks key-word which needs a list of marks (marks=['#1', '#2'])

        
  'followed by integer\n' + self.mark_tree.__doc__)


p-value of positive selection for sites on this branch is:
1
p-value of relaxation for sites on this branch is:
0.014542934568084975
we have relaxation on sites on this branch

clean tree, remove marks

---------
Now working with leaf Pan_troglodytes
((Hylobates_lar ,(Gorilla_gorilla ,Pan_troglodytes #1) ) ,Papio_cynocephalus );
running model bsA and bsA1


/Users/jra/miniconda3/envs/ete3/lib/python3.6/site-packages/ete3/evol/evoltree.py:540: UserWarning: 
Large differences (> 0.1) may indicate mistaken assigantion of null and alternative models
  "\nLarge differences (> 0.1) may indicate mistaken "


p-value of positive selection for sites on this branch is:
1
p-value of relaxation for sites on this branch is:
0.001295136738279079
we have relaxation on sites on this branch

clean tree, remove marks

---------
Now working with leaf Papio_cynocephalus
((Hylobates_lar ,(Gorilla_gorilla ,Pan_troglodytes ) ) ,Papio_cynocephalus #1);
running model bsA and bsA1


/Users/jra/miniconda3/envs/ete3/lib/python3.6/site-packages/ete3/evol/evoltree.py:540: UserWarning: 
Large differences (> 0.1) may indicate mistaken assigantion of null and alternative models
  "\nLarge differences (> 0.1) may indicate mistaken "


p-value of positive selection for sites on this branch is:
0.7498870274777537
p-value of relaxation for sites on this branch is:
0.013438319350932315
we have relaxation on sites on this branch

clean tree, remove marks
 Evolutionary Model bsA.Hylobates_lar:
        log likelihood       : -521.20329
        number of parameters : 11
        sites inference      : BEB, NEB
        sites classes        : 
        background w: b0=0.06451   b1=1.0       b2=0.06451   b3=1.0       
        foreground w: f0=0.06451   f1=1.0       f2=3.16388   f3=3.16388   
        proportions : p0=0.0       p1=0.0       p2=0.94828   p3=0.05172   
        site class  :  
        branches             : 
        mark: #0  , omega: None      , node_ids: 5   , name: ROOT
        mark: #0  , omega: None      , node_ids: 6   , name: EDGE
        mark: #0  , omega: None      , node_ids: 4   , name: Papio_cynocephalus
        mark: #1  , omega: None      , node_ids: 2   , name: Hylobates_lar
        mark: #0  , omega: